# Working with supplier images

changeImage.py

In [ ]:
#!/usr/bin/env python3

import os
from PIL import Image

path = 'supplier-data/images/'

for image in os.listdir(path):
    if image.endswith(".tiff"):
        
        file_name = os.path.splitext(image)[0]
        
        try:
            with Image.open(path + image).convert("RGB") as im:
                im.thumbnail((600, 400))
                im.save(path + file_name + '.jpeg', "JPEG")
        except OSError:
            print('can not convert {}'.format(image))

            

# Uploading images to web server

example_upload.py ,  supplier_image_upload.py

In [ ]:
# example_upload.py


#!/usr/bin/env python3
import requests

# This example shows how a file can be uploaded using
# The Python Requests module

url = "http://localhost/upload/"
with open('/usr/share/apache2/icons/icon.sheet.png', 'rb') as opened:
    r = requests.post(url, files={'file': opened})


# This example shows how a file be uploaded by using the Python 'requests' module

In [ ]:
# supplier_image_upload.py

#!/usr/bin/env python3
import requests
import os

path = 'supplier-data/images/'
url = "http://localhost/upload/"

for image in os.listdir(path):
    if "jpeg" in image:
        with open(path + image, 'rb') as im:
            r = requests.post(url, files = {'file':im})


# Uploading the descriptions

run.py

In [ ]:
#!/usr/bin/env python3
import os
import json
import requests

path = 'supplier-data/descriptions'
url = "http://localhost/fruits/"
description = {}

for file in os.listdir(path):

    if file.endswith("txt"):
        file_path = os.path.join(path, file)
        file_name = os.path.splitext(file)[0]
        with open(file_path, 'r') as file_info:
            file_lines = file_info.readlines()
            description['name'] = file_lines[0].rstrip('\n')
            description['weight'] = int(file_lines[1].rstrip('\n').strip(" lbs"))
            description['description'] = file_lines[2].rstrip('\n')
            description['image_name'] = file_name + '.jpeg'

        print(description)
        
        response = requests.post(url, json=description)
        print(response.text)
        
        response.raise_for_status()
        print(response.ok)
        print(response.status_code)
        print(response.request.url)
        print(response.request.body)


    

In [ ]:
#!/usr/bin/env python3
import os, sys
import json
import requests

path = "supplier-data/descriptions/"
url = "http://localhost/fruits/"

files = os.listdir(path)

for file in files:
  if file.endswith("txt"):
    with open(path + file, 'r') as f:
      #grab the file name, ex. 001, 002 to use for image file
      fruit_name = os.path.splitext(file)[0]
      data = f.read()
      data = data.split("\n")
      fruit_dic = {"name": data[0], "weight": int(data[1].strip(" lbs")), "description": data[2], "image_name": fruit_name + ".jpeg"}
      response = requests.post(url, json=fruit_dic)
      response.raise_for_status()
      print(response.request.url)
      print(response.status_code)


# Generate a PDF report and send it through email

reports.py , report_email.py , emails.py

In [ ]:
# reports.py
# to generate PDF report to supplier

#!/usr/bin/env python3

from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Paragraph, Spacer, Table, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# generate the pdf file 
def generate_report(filename, title, additional_info):
    styles = getSampleStyleSheet()
    report = SimpleDocTemplate(filename)
    report_title = Paragraph(title, styles["h1"])
    report_info = Paragraph(additional_info, styles["BodyText"])
    empty_line = Spacer(1,20)
    
    report.build([report_title, empty_line, report_info, empty_line])



In [ ]:
# report_email.py

#!/usr/bin/env python3
import os
import reports
import datetime
import smtplib
import emails

# to get the important info from .txt file, return a sting of important info 
def get_pdf_info(path):
    return_info = ''
    for file in os.listdir(path):
        with open(path + file, 'r') as file_info:
            info = file_info.readlines()
            name = info[0].rstrip('\n')
            weight = info[1].rstrip('\n')
            
            return_info += "<br/>" + "name: " + name + "<br/>" + "weight: " + weight + "<br/>"
            
    return return_info


def main():
    path = 'supplier-data/descriptions/'
    
    # prepare input for the "reports.generate_report"
    today_date = datetime.datetime.now().date()
    today_date = today_date.strftime('%b %d,%Y')
    title = 'Processed Update on ' + today_date
    fruit_report = get_pdf_info(path)
    
    # create the pdf 
    reports.generate_report('/tmp/processed.pdf', title, fruit_report)
    
    # generate the email
    message = emails.generate_email('automation@example.com',
                                    "{}@example.com".format(os.environ.get('USER')),
                                    'Upload Completed - Online Fruit Store', 
                                    'All fruits are uploaded to our website successfully. A detailed list is attached to this email.', 
                                    '/tmp/processed.pdf')
    
    # send the email
    emails.send_email(message)
    
    
if __name__ == "__main__":
    main()

In [ ]:
# emails.py

#!/usr/bin/env python3

import email.message
import mimetypes
import os.path
import smtplib

                                                    # attachment_path = None !!!! super important
                                                    # avoid the problem that we always have to have an attachment
def generate_email(sender, recipient, subject, body, attachment_path = None):
    
    # Basic Email formatting
    message = email.message.EmailMessage()
    message["From"] = sender
    message["To"] = recipient
    message["Subject"] = subject
    message.set_content(body)  # create the body part 
    
    if attachment_path != None
        # Process the attachment and add it to the email
        attachment_filename = os.path.basename(attachment_path)
        mime_type, _ = mimetypes.guess_type(attachment_path)
        mime_type, mime_subtype = mime_type.split('/', 1)

        with open(attachment_path, 'rb') as ap:
            message.add_attachment(ap.read(),
                                  maintype=mime_type,
                                  subtype=mime_subtype,
                                  filename=attachment_filename)

    return message 


# send email function
def send_email(message):
    # """Sends the message to the configured SMTP server."""
    mail_server = smtplib.SMTP('localhost')
    mail_server.send_message(message)
    mail_server.quit()



# Health check

This is the last part of the lab, where you will have to write a Python script named health_check.py that will run in the background monitoring some of your system statistics: CPU usage, disk space, available memory and name resolution. Moreover, this Python script should send an email if there are problems, such as:



- Report an error if CPU usage is over 80%
- Report an error if available disk space is lower than 20%
- Report an error if available memory is less than 500MB
- Report an error if the hostname "localhost" cannot be resolved to "127.0.0.1"

health_check.py

In [ ]:
#!/usr/bin/env python3 

import os, sys
import psutil 
import shutil
import socket
import emails


# check CPU usage (Error, if cpu usage > 80%)
# return 'True' if error occurs
def cpu_check():
    cpu_usage = psutil.cpu_percent(1)
    cpu_error = cpu_usage > 80
    return cpu_error

# check available disk space
# return 'True' if error occurs
def disk_space_check():
    disk_space = shutil.disk_usage("/")  # output look like this: usage(total=126753091584, used=99699490816, free=27053600768)
    disk_total = disk_space.total
    disk_free = disk_space.free
    available_space = (disk_free / disk_total) * 100
    disk_space_error = available_space < 20
    return disk_space_error

# check available memory
# return 'True' if error occurs
def memory_check():
    available_memory = psutil.virtual_memory()  # output look like this: vmem(total=8374149120L, available=1247768576L)
    #convert to MB ( / 1024 ** 2)
    available_in_MB = available_memory.available >> 20
    memory_error = available_in_MB < 500
    return memory_error

# check the hostname
# return 'True' if error occurs
def hostname_check():
    hostname = socket.gethostbyname('localhost')
    hostname_error = hostname != "127.0.0.1"
    return hostname_error

# send warning emails function
def warning_emails(error):
    sender = "automation@example.com"
    receiver = "{}@example.com".format(os.environ["USER"])
    subject = error
    body = "Please check your system and resolve the issue as soon as possible."
    message = emails.generate_email(sender, receiver, subject, body)
    emails.send_email(message)

# run the error check
list_of_check = [cpu_check(), disk_space_check(), memory_check(), hostname_check()]
list_of_error = ['Error - CPU usage is over 80%  ',
                'Error - Available disk space is less than 20%  ',
                'Error - Available memory is less than 500MB  ',
                'Error - localhost cannot be resolved to 127.0.0.1  ']

error_message = ''
if True in list_of_check:
    for i in range(len(list_of_check)):
        check_error = list_of_check[i]
        if check_error == True:
            error_message += list_of_error[i]

    # send warning emails
    warning_emails(error_message)